In [ ]:
%matplotlib inline

import matplotlib.pylab as plt 
import numpy as np
import os
import warnings

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from obspy import *
from obspy.clients.fdsn import Client
from obspy.core import read, UTCDateTime
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from os import path

from plot_event_station import plot_event_station
from plotw_rs import plotw_rs

warnings.filterwarnings('ignore')                  # do not show warnings

# Applied Seismology, GEOS 626, University of Alaska Fairbanks
# Python coding by Nealey Sims
#
# Template script for analyzing direct arrival waveforms from Sumatra:
#    channel LHZ, duration up to 10 days
#
# This assumes you have previously run lab_sumatra.ipynb to save a set of waveforms
# in the local directory ./datawf/
#
# Calls plotw_rs

# Plotting parameter
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

In [ ]:
spdy = 86400;   # seconds per day
# extract the full database of BHZ waveforms

otimePDE = UTCDateTime("2004-12-26T00:58:53.0")   # origin time of Sumatra earthquake
originTime = otimePDE;
startTime  = originTime - .5*spdy
endTime    = originTime + 9*spdy
elat = 3.09
elon = 94.26
LHdirectory = "./datawf/sumatraLH"
client=Client("IRIS")
datadir="./data"
iprocess = True       #deconvolve instrument response if True

In [ ]:
# max radius for station search
stasub = [0,180] #in degrees

stanames=[]
# Load pre-saved BHZ stations 
w=Stream()
wc=read(LHdirectory+"/*Z", header=None)
#print(wc)
wc.merge(method=1, fill_value=0)
inventory = client.get_stations(minradius=stasub[0], maxradius=stasub[1], longitude=elon, 
                    latitude=elat, network="G,II,IU", starttime=startTime, endtime =endTime,  
                    channel="LHZ", level="response")

print(inventory)
for net in inventory:     
    for sta in net:
        for cha in sta:
            if str(sta.code)+str(cha.location_code) not in stanames:
                
                # Use inventory to save station locations
                try:
                    ws=wc.select(station=sta.code, location=cha.location_code)
                    ws[0].trim(originTime-(5*60),originTime+(2*3600), pad=True, fill_value=0)
                    ws[0].stats.sampling_rate=round(ws[0].stats.sampling_rate)
                    ws[0].stats.sac.stla=sta.latitude
                    ws[0].stats.sac.stlo=sta.longitude
                    if iprocess == True:
                        ws[0].remove_response(inventory=inventory, water_level=20)
                    w=w.append(ws[0])
                    stanames.append(str(sta.code)+str(cha.location_code))
                except:
                    pass                
# Add station location data to the Obspy Trace
nw=len(w)
print('here is a list of the waveforms you have:');
for ii in range (nw):
    print('%3i %7s %3s %6s' % (ii,w[ii].stats.channel,w[ii].stats.network,w[ii].stats.station))

# save a copy to avoid rerunning
wpick=Stream()
# pick a subset of waveforms
#ipick = np.arange(nw)                   # default
ipick = [36,1,61,49,50,159]     # USER: CHANGE THIS 
for i in ipick:
    wpick.append(w[i])


In [ ]:
# PLOTTING PARAMETERS FOR plotw_rs.m (USER: CHANGE THESE AS NEEDED)
rssort = 2      # =1 by azimuth, =2 by distance
iabs = 0
tshift = []
tmark = []
pmax = 40
iintp = 0
inorm = [1]
tlims = []
nfac = 1
azstart = []
iunit = 2
imap = 0

# plot record section
T1 = [100];
T2 = [1000];
# USER: add necessary arguments if they are different from the default
plotw_rs(wpick,elat,elon,rssort,iabs,tshift,tmark,T1,T2,pmax,iintp,inorm,tlims,
                 nfac,azstart,iunit,imap)

In [ ]:
# Plot global map of selected stations
plot_event_station(elat,elon,w=wpick)